In [7]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series
from datetime import datetime
from copy import deepcopy
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Import dataset
aurora_cara = pd.read_csv("Aurora v2.1 data file - caracterización.csv")
aurora_feedback = pd.read_csv("Aurora v2.1 data file - ayudaHumanitaria.csv")
aurora_monitoreo = pd.read_csv("Aurora v2.1 data file - monitoreo.csv")

In [8]:
aurora = pd.merge(aurora_cara, aurora_feedback)
aurora.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 736 entries, 0 to 735
Data columns (total 70 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   ¿Quién entra al sistema?                                736 non-null    object 
 1   ¿Cómo interactúa con el sistema?                        736 non-null    object 
 2   Tipo de enganche                                        736 non-null    object 
 3   UserId                                                  736 non-null    int64  
 4   InteraID                                                736 non-null    object 
 5   Inicio interacción                                      736 non-null    object 
 6   Final interacción                                       0 non-null      float64
 7   Tiempo de interacción                                   225 non-null    float64
 8   Alguien te está ayudando a conectarte   

In [9]:
aurora_comple= aurora.append(aurora_monitoreo)

C:\Users\maola\AppData\Local\Temp\ipykernel_6612\1469997505.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aurora_comple= aurora.append(aurora_monitoreo)


In [10]:
# Drop observations of Aurora team phones and test registers 
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311571598].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311398466 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311396734 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311361421 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311361350 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311361257 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311337494 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311325070 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311325038 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311272934 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 310820267 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 310543580 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 310357249 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 310191611 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 308421831 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 306028996 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311725039].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311719001 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311718121].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311699383 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311696700 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 312179120 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311965863 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311965863 ].index)

In [11]:
#Variable of date in date format
aurora_comple["Inicio interacción"] = pd.to_datetime(
aurora_comple["Inicio interacción"], format='%Y-%m-%d %H:%M:%S', errors='coerce', utc=True).dt.strftime('%Y-%m-%d')
#aurora = aurora[aurora["Inicio interacción"] < '2023-10-29']

In [12]:
aurora_comple= aurora_comple.sort_values(by=['UserId', 'Inicio interacción'])

# Reset the index for proper grouping
aurora_comple= aurora_comple.reset_index(drop=True)

# Create a new variable "Interaction_Sequence" based on the grouping
aurora_comple['Interaction_Sequence'] = aurora_comple.groupby('UserId').cumcount() + 1

In [13]:
aurora_comple['Edad'] = aurora_comple.groupby('UserId')['Edad'].transform('first')

In [14]:
excel_file = 'completa.xlsx'
aurora_comple.to_excel(excel_file, index=False)

In [31]:
unpivoted = aurora_comple.groupby(['Inicio interacción', 'Tipo de monitoreo'])[
    'UserId'].count().reset_index()
pivoted = unpivoted.pivot(
    columns='Inicio interacción',
    index=[ 'Tipo de monitoreo'],
    values='UserId')
# Reset the index of the pivoted DataFrame
pivoted = pivoted.reset_index()
pivoted

Inicio interacción,Tipo de monitoreo,2023-10-18,2023-10-19,2023-10-20,2023-10-21,2023-10-22,2023-10-23,2023-10-24,2023-10-25,2023-10-26,...,2023-11-02,2023-11-03,2023-11-04,2023-11-06,2023-11-07,2023-11-08,2023-11-09,2023-11-10,2023-11-11,2023-11-12
0,Feedback,NaN,NaN,3.0,2.0,4.0,6.0,7.0,7.0,7.0,...,22.0,15.0,23.0,20.0,19.0,22.0,6.0,16.0,40.0,27.0
1,NNA,1.0,NaN,2.0,5.0,4.0,2.0,NaN,4.0,3.0,...,NaN,4.0,9.0,1.0,1.0,17.0,16.0,9.0,5.0,6.0
2,Percepción,1.0,1.0,3.0,NaN,6.0,1.0,1.0,2.0,4.0,...,10.0,4.0,6.0,6.0,9.0,2.0,7.0,9.0,16.0,3.0
3,Vocación Permanencia,2.0,4.0,1.0,2.0,2.0,NaN,NaN,2.0,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
excel_file = 'monitoreos.xlsx'
pivoted.to_excel(excel_file, index=False)

In [69]:
# Rename variables 

newColumns = {
    ' Principales necesidades de información' : 'neces_info',

}

aurora_comple = aurora_comple.rename(columns=newColumns)

In [70]:
# Necesidades de información separar 
aurora_comple['neces_info'] = aurora_comple['neces_info'].str.split('|')
prueba = aurora_comple.explode('neces_info')

In [71]:
# Use dictionary to give other values to the variable
info_dict = {
1 :  'Acceso a alberge o alojamiento temporal',
2 : 'Acceso a alimentación o kit de alimentos',
3 : 'Acceso a ropa u otros artículos de cuidado personal',
4 : 'Acceso a atención en salud y medicamentos',
5 : 'Acceso a asistencia legal',
6 : 'Ubicación de la ayuda humanitaria en la ruta',
7 : 'Seguridad en la ruta y lugares de destino',
8 : 'Información sobre politica migratoria y procesos de regularización',
9 : 'Otra'
}

def Map(value):
    try:

        if (type(value) == int):
            return info_dict[value]
        if (type(value) == str):
            return info_dict[int(value)]
    except Exception as e:
        return value


prueba['neces_info'] = prueba['neces_info'].apply(lambda x: Map(x))

In [73]:
prueba = prueba[['UserId',
               'Edad',
               'Género',
               'Lugar interacción',
               'neces_info',
               'A través de qué medios a recibido información',

               ]].reset_index()

In [74]:
excel_file = 'info.xlsx'
prueba.to_excel(excel_file, index=False)

In [ ]:
columns = aurora_carto.columns.tolist()

In [ ]:
for column in columns:
  sns.countplot(x=column, data=aurora_carto, order = aurora_carto[column].value_counts().index)
  plt.xticks(rotation=70, fontsize=10)
  plt.xlabel(column, fontsize=12)
#  plt.title(f'[{column}]', fontsize=12)
  plt.show()
  plt.clf()